## Collect Manhattan Real Estate Data
This is a workflow for using the ```zillow_scraper``` package (https://github.com/hansenrhan/zillow_scraper) to collect rental and sales data in Manhattan in 2024. The ultimate goal is to do comparisons between rental vs. buying apartments in Manhattan, and evaluate how buying property in Manhattan compares to other possible investments such index funds.

Author: Hansen Han    
Date: April 7, 2024

### Load Packages, Set Config

In [ ]:
# Load packages
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import math
import re
import time
import urllib.parse
from datetime import datetime
from zillow_scraper import zillow_scraper, make_frame_sales, make_frame_rentals, make_frame_rentals_detail
from selenium import webdriver
from tqdm import tqdm

# set config
warnings.flterwarnings('ignore')
today = datetime.now()
todays_date = today.strftime("%Y_%m_%d")
todays_date

### Define Helper Functions

In [ ]:
# helper functions
def extract_zillow_page_json_modified(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Locate the <script> tag with the specified id
    script_tag = soup.find('script', {'id': '__NEXT_DATA__'})

    data = None
    # Extract the JSON content (if the tag is found)
    if script_tag:
        json_content = script_tag.string
        # Now, you can parse the json_content using json.loads
        data = json.loads(json_content)
    
    return data

def average_time_between_dates(date_strings):
    # Convert string dates to datetime objects
    dates = [datetime.strptime(date_str, '%Y-%m-%d') for date_str in date_strings]

    # Calculate time differences between consecutive dates
    time_diffs = [dates[i+1] - dates[i] for i in range(len(dates)-1)]

    # Calculate total time difference and number of differences
    total_time_diff = sum(time_diffs, datetime.min - datetime.min)
    num_diffs = len(time_diffs)

    # Calculate average time difference
    average_time_diff = total_time_diff / num_diffs

    return average_time_diff

def total_years_between_dates(date_strings):
    # Convert string dates to datetime objects
    dates = [datetime.strptime(date_str, '%Y-%m-%d') for date_str in date_strings]

    # Find the minimum and maximum dates
    min_date = min(dates)
    max_date = max(dates)

    # Calculate the difference in years between the minimum and maximum dates
    total_years = (max_date - min_date).days / 365.25

    return total_years

import math

def calculate_cagr(initial_value, final_value, periods):
    """
    Calculate Compound Annual Growth Rate (CAGR)
    
    Parameters:
        initial_value (float): Initial value
        final_value (float): Final value
        periods (int): Number of periods
        
    Returns:
        float: Compound Annual Growth Rate (CAGR)
    """
    cagr = (final_value / initial_value) ** (1 / periods) - 1
    return cagr


def sales_stats_calculator(price_data):
    # calculate price history data of interest
    # total sales events
    # average time between sales
    # appreciation 
    # total return
    # time range
    # stdev_price
    sales_events = None
    try:
        
        # we only want to see sales
        price_data = price_data[price_data.event == "Sold"]
        price_data = price_data.reset_index(drop=True)

        sales_events = len(price_data['event'])
        
        # if we have less than 2 events, we can't calculate stats
        if sales_events < 2:
            return sales_events, None, None, None, None, None

        # calculate the average time between sales
        dates = price_data['date']
        average_time_between_sales = average_time_between_dates(dates)

        # calculate cumulative appreciation 
        time_range = total_years_between_dates(dates)

        # calculate maximum drawdown (lowest point relative to initial)
        earliest_sale_price = price_data['price'][len(price_data) - 1]
        lowest_price = earliest_sale_price
        for sale_price in list(price_data['price']):
            if sale_price < lowest_price:
                lowest_price = sale_price
        
        max_drawdown = round(lowest_price/earliest_sale_price, 2) - 1

        # calculate total return
        latest_sale_price = price_data['price'][0]
        total_return = latest_sale_price/earliest_sale_price - 1
        annual_return_raw = total_return/time_range

        # calculate compound annual growth rate(CAGR)
        cagr = calculate_cagr(earliest_sale_price, latest_sale_price, time_range)

        return sales_events, average_time_between_sales.days / 365.25, time_range, max_drawdown, total_return, cagr
    except Exception as e:
        print("Error:", e)
        traceback.print_exc()
        return sales_events, None, None, None, None, None


def get_hoa_fee_from_html(html):
    try:
        # Initialize a BeautifulSoup object
        soup = BeautifulSoup(html, 'html.parser')

        # Find the span containing "monthly HOA fee"
        target_span = soup.find('span', string=re.compile(r'HOA fee'))

        if target_span:
            
            text_content = target_span.string
            if text_content:
                text_content = text_content.replace("$", "").replace(",", "").replace("HOA fee:", "").replace("monthly", "").replace(" ", "")
                number = float(text_content)
            # Extract the number from the string
            #number_match = re.search(r'\$(\d+)', text_content)
            
            if number:
                #number = int(number_match.group(1))
                return number
            else:
                return None
        else:
            return None

    except Exception as e:
        print(e)
        return None


def generate_capitalization_variations(string, index, current_variation, all_variations):
    if index == len(string):
        all_variations.append(current_variation)
        return
    
    char = string[index]
    
    # Lowercase variation
    generate_capitalization_variations(string, index + 1, current_variation + char.lower(), all_variations)
    
    # Uppercase variation
    generate_capitalization_variations(string, index + 1, current_variation + char.upper(), all_variations)


def extract_building_type(building_num, street):
    search_query = "1|{building_num}|{street}|1".format(building_num = building_num, street=street)
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Accept-Language": "en-US,en;q=0.9",
        }


        url = "https://a810-dobnow.nyc.gov/Publish/WrapperPP/PublicPortal.svc/getPublicPortalPropertyDetailsGet/{search_query}".format(search_query=quote(search_query))
        r = requests.get(url, headers=headers)
        prop_dict = json.loads(r.text)
        return prop_dict['PropertyDetails']['VlFinaOccpncy']
    except Exception as e:
        print("Error in extract_builing_type():", e)
        return None

### Collect Current Rental & Sales Listings for Manhattan

In [ ]:
property_types = ["sale", "rent"]
location = "manhattan-ny"

for property_type in property_types:
    data_dict = zillow_scraper(city=location, property_type=property_type, time_between_scrapes=120, testing=False, min_price=0)

    data = data_dict["data_list"]
    min_price = data_dict["min_price"]
    num_listings = data_dict["num_listings"]
    df = pd.DataFrame()

    # sales properties need to be handled slightly different...
    if property_type == "sale":
        # extract into a table
        df = make_frame_sales(df, data)

        #drop cols
        #df = df.drop('hdpData', 1) #remove this line to see a whole bunch of other random cols, in dict format

        #drop dupes
        df = df.drop_duplicates(subset='zpid', keep="last")

        #filters
        df['zestimate'] = df['zestimate'].fillna(0)
        df['best_deal'] = df['unformattedPrice'] - df['zestimate']
    else:
        df = make_frame_rentals_detail(df, data)
    
    df.to_csv("data/{location}_{property_type}_{date}.csv".format(location=location, property_type=property_type, date=todays_date))

### Collect Historical Sales Data

#### Collect Additional Sale Listing Information (HOA Fee, Historical Sales)

In [ ]:
# Load Sales Data
sales_data = pd.read_csv("manhattan-ny_sale_2024_03_15.csv")
sales_data

In [ ]:
from zillow_scraper import get_hoa_fee

sales_events_results = []
avg_years_between_sales_results = []
total_range_results = []
max_drawdown_results = []
total_return_results = []
cagr_results = []
hoa_fees = []
rent_zestimates = []

for url in tqdm(sales_data['detailUrl']):
    html_content = None
    try:
        driver = webdriver.Chrome()
        # open the page
        driver.get(url)

        html_content = driver.page_source
        driver.quit()
    except Exception as e:
        print("Error:", e)
        pass

    data = extract_zillow_page_json_modified(html_content)

    sub_data = json.loads(data['props']['pageProps']['componentProps']['gdpClientCache'])

    # extract and calculate information about property sales
    price_history_df = pd.DataFrame(sub_data[list(sub_data.keys())[0]]['property']['priceHistory'])
    sales_events, avg_years_between_sales, total_range, max_drawdown, total_return, cagr = sales_stats_calculator(price_history_df)
    sales_events_results.append(sales_events)
    avg_years_between_sales_results.append(avg_years_between_sales)
    total_range_results.append(total_range)
    max_drawdown_results.append(max_drawdown)
    total_return_results.append(total_return)
    cagr_results.append(cagr)

    # record HOA fees
    try:
        hoa_fee = float(sub_data[list(sub_data.keys())[0]]['property']['resoFacts']['hoaFeeTotal'].replace("$", "").replace(",", "").replace(" monthly", ""))
    except:
        hoa_fee = None
    hoa_fees.append(hoa_fee)

    # record rental estimate from zillow
    try:
        rent_zestimate = float(sub_data[list(sub_data.keys())[0]]['property']['rentZestimate'])
    except:
        rent_zestimate = None
    rent_zestimates.append(rent_zestimate)


    #time.sleep(45) # sleep for 45 seconds

sales_data['sales_events'] = sales_events_results
sales_data['avg_years_between_sales'] = avg_years_between_sales_results
sales_data['total_range_sales'] = total_range_results
sales_data['max_sales_loss'] = max_drawdown_results
sales_data['total_return'] = total_return_results
sales_data['historical_cagr'] = cagr_results
sales_data['hoa_fee'] = hoa_fees
sales_data['rent_zestimate'] = rent_zestimates

sales_data

### Get Building Type (Condo or Co-Op?)

In [ ]:
# Generate an array of combinations of potential prefixes to an address to handle
# We just want the building address, so we need to remove any possible extra information
# ...otherwise we won't be able to programatically search from the Department of Buildings...

split_chs = ["#", "Unit", "Apt", "Penthouse", "Floor", "Room", "Ph", "Suite", "Front"]
all_variations = []

for ch in split_chs:
    generate_capitalization_variations(ch, 0, "", all_variations)


split_chs = [" " + x for x in all_variations]
split_chs

In [ ]:
# only get building types on buildings with sales data (save time and requests)
# if we have at least 2 points, we have points which we can calculate return over time
filtered_sales_data = sales_data[sales_data.sales_events >= 2] #
filtered_sales_data

In [ ]:
# search up apartment type from the DOB

building_types = []

addresses = filtered_sales_data['addressStreet']
for address in tqdm(addresses):
    building_num = address.split(" ")[0]
    address = address.replace(building_num + " ", "")
    for x in split_chs:
        if x in address:
            address = address.split(x)[0]
            break
    
    building_types.append(extract_building_type(building_num, address))

building_types

In [ ]:
# save data
filtered_sales_data.to_csv("data/manhattan-ny_sale_2024_03_15_with_cagr_and_hoa_fees.csv")